### Initiate llm

In [3]:
#from langchain.llms import OpenAI
#from dotenv import load_dotenv

In [7]:
load_dotenv()

True

In [8]:
llm = OpenAI(temperature=0.6, max_tokens=500)

In [15]:
# Test LLM

In [9]:
essay = llm("write email requesting refund for electronic item")
print(essay)



Subject: Request for Refund on electronically purchased item

Dear [Name],

I am writing to request a refund for an electronically purchased item.

On [date], I purchased [item name] from [website/app]. Unfortunately, the item was defective upon delivery and I am now requesting a refund.

I understand that I may need to return the item to you before I receive a refund. If this is the case, please provide information on how I should return the item.

I have included my proof of purchase and order number [include order number] below for your reference.

Thank you for your time and consideration.

Sincerely, 
[Your Name]


### Loading data

In [11]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path= './data/codebasics_faqs.csv', source_column="prompt")

# Store the loaded data in the 'data' variable
data = loader.load()

In [26]:
data

75

### Embbedding

In [13]:
from langchain.embeddings import OpenAIEmbeddings

In [14]:
embeddings = OpenAIEmbeddings()

### Split data into chunks

In [17]:
from langchain.text_splitter import CharacterTextSplitter

In [21]:
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

In [22]:
docs = text_splitter.split_documents(data)

In [24]:
len(docs)

75

### Vector database

In [25]:
from langchain.vectorstores import FAISS

In [27]:
db = FAISS.from_documents(docs, embeddings)

In [ ]:
# Test for similarity

In [29]:
query = "Can someone help me to create same chart exactly like this. I have created more than half of it. I want to add quadrant labels. Low medium high on x & y axis"
docs = db.similarity_search(query)

In [30]:
print(docs[0].page_content)

prompt: Can someone help me to create same chart exactly like this. I have created more than half of it. I want to add quadrant labels. Low medium high on x & y axis
response: Check this thread for instructions : https://discord.com/channels/1090613684163850280/1107745807848980490/1107746891418058762


In [31]:
# Create a retriever for querying the vector database
retriever = db.as_retriever(score_threshold = 0.7)

In [33]:
rdocs = retriever.get_relevant_documents("I’m not sure if this course is good enough for me to invest some money. What can I do?")
len(rdocs)

4

### RetrievalQA

In [34]:
from langchain.chains import RetrievalQA

In [35]:
#when retrieving data some hallucinations occur, we need to avoid this.

In [36]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [37]:
chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff", #map-reduced
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs) 

In [38]:
chain('Do you provide job assistance and also do you provide job gurantee?')

{'query': 'Do you provide job assistance and also do you provide job gurantee?',
 'result': '\n\nANSWER: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters. We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.',
 'source_documents': [Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation alo

In [39]:
chain("Do you guys provide internship and also do you offer EMI payments?")

{'query': 'Do you guys provide internship and also do you offer EMI payments?',
 'result': '\n\nANSWER: Yes, we provide virtual internship opportunities. However, we do not offer EMI payments.',
 'source_documents': [Document(page_content='prompt: Do you provide any virtual internship?\nresponse: Yes', metadata={'source': 'Do you provide any virtual internship?', 'row': 14}),
  Document(page_content='prompt: Do we have an EMI option?\nresponse: No', metadata={'source': 'Do we have an EMI option?', 'row': 13}),
  Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
  Document(page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube avail